In [7]:
import pandas as pd

# Loading Datasets
eda = pd.read_csv("../data/processed/eda_shipping_features.csv")
preds = pd.read_csv("../data/processed/model_predictions_inventory.csv")
forecast = pd.read_csv("../data/processed/nike_inventory_forecast.csv")

forecast.rename(columns={"ds": "date"}, inplace=True)

# Merging Datasets
merged = (
    eda.merge(preds, on="date", how="outer")
    .merge(forecast, on="date", how="outer")
    .sort_values("date")
)

# Export dataset for Tableau
merged.to_csv("../data/processed/nike_supplychain_tableau_final.csv", index=False)
print("Tableau dataset saved")
print("Merged shape: ", merged.shape)

Tableau dataset saved
Merged shape:  (129, 14)


In [3]:
import pandas as pd

# Loading cleaned dataset
df = pd.read_csv("../data/processed/nike_supplychain_tableau_final.csv")

# Selecting relevent columns
cols = ["freight_cost_index", "oil_price_usd", "port_congestion_index"]

# Compute correlation
corr = df[cols].corr()

# Saving as csv
corr.to_csv("../data/processed/nike_correlation_matrix.csv")

corr

,freight_cost_index,oil_price_usd,port_congestion_index
freight_cost_index,1.000000,-0.605221,0.639670
oil_price_usd,-0.605221,1.000000,-0.592496
port_congestion_index,0.639670,-0.592496,1.000000


In [3]:
import pandas as pd
import numpy as np

# Load your existing cleaned dataset
df = pd.read_csv("../data/processed/nike_supplychain_tableau_final.csv")

# Create a list of Nike's key operating regions
regions = ["North America", "Europe", "Asia-Pacific", "Latin America"]

# Assign regions in a repeating pattern
df["region"] = [regions[i % len(regions)] for i in range(len(df))]

# Save it as the new version
df.to_csv("../data/processed/nike_supplychain_tableau_final_with_region.csv", index=False)

print("✅ Added region column:", df["region"].unique())
print("✅ File saved as nike_supplychain_tableau_final_with_region.csv")


✅ Added region column: ['North America' 'Europe' 'Asia-Pacific' 'Latin America']
✅ File saved as nike_supplychain_tableau_final_with_region.csv


In [5]:
import pandas as pd
import numpy as np

# Load your regional dataset
df = pd.read_csv("../data/processed/nike_supplychain_tableau_final_with_region.csv")

# --- INVENTORY DAYS ---
# Base around port congestion and freight cost
df["inventory_days"] = (
    30
    + (df["port_congestion_index"] * 2)
    + (df["freight_cost_index"] * 0.5)
    + np.random.normal(0, 2, len(df))  # add small noise
)

# --- REVENUE USD ---
# Base around consumer demand and oil/freight cost
df["revenue_usd"] = (
    500_000
    + (df["consumer_demand_index"] * 10_000)
    - (df["freight_cost_index"] * 300)
    + np.random.normal(0, 10_000, len(df))  # small random variation
)

# --- SAVE UPDATED FILE ---
df.to_csv("../data/processed/nike_supplychain_tableau_final_enriched.csv", index=False)

print("✅ Added Nike business metrics:")
print(df[["region", "inventory_days", "revenue_usd"]].head())


✅ Added Nike business metrics:
          region  inventory_days    revenue_usd
0  North America      957.343820  900830.098333
1         Europe      885.526665  880985.028281
2   Asia-Pacific      836.590953  856004.786877
3  Latin America      963.948166  802607.811184
4  North America      840.611562  823842.226646


In [6]:
import pandas as pd
import numpy as np

# Load your dataset
df = pd.read_csv("../data/processed/nike_supplychain_tableau_final_with_region.csv")

# Base formula: freight + 0.5 * oil + 0.3 * congestion + random variation
np.random.seed(42)
df["shipping_cost"] = (
    df["freight_cost_index"] * 1.5
    + df["oil_price_usd"] * 0.8
    + df["port_congestion_index"] * 0.3
    + np.random.normal(0, 10, len(df))
)

# Round for readability
df["shipping_cost"] = df["shipping_cost"].round(2)

# Save back to CSV
df.to_csv("../data/processed/nike_supplychain_tableau_final_with_region.csv", index=False)

print("✅ Added shipping_cost column successfully!")
print(df[["date", "region", "revenue_usd", "shipping_cost"]].head())


✅ Added shipping_cost column successfully!


KeyError: "['revenue_usd'] not in index"

In [7]:
import pandas as pd

df = pd.read_csv("../data/processed/nike_supplychain_tableau_final_with_region.csv")
print(df.columns)


Index(['date', 'freight_cost_index', 'oil_price_usd', 'port_congestion_index',
       'consumer_demand_index', 'freight_cost_rolling', 'oil_price_rolling',
       'event', 'actual_inventory', 'predicted_inventory',
       'ridge_predicted_inventory', 'yhat', 'yhat_lower', 'yhat_upper',
       'region', 'shipping_cost'],
      dtype='object')


In [8]:
import numpy as np
import pandas as pd

# Load data again
df = pd.read_csv("../data/processed/nike_supplychain_tableau_final_with_region.csv")

# --- Generate synthetic Nike revenue (USD) ---
np.random.seed(42)
base_revenue = 5000  # baseline revenue in millions
df["revenue_usd"] = (
    base_revenue
    + (df["consumer_demand_index"] * 15)  # tied to demand
    - (df["shipping_cost"] * 5)           # higher cost → lower revenue
    + np.random.normal(0, 50, len(df))    # noise
)

df["revenue_usd"] = df["revenue_usd"].round(2)

# Save again
df.to_csv("../data/processed/nike_supplychain_tableau_final_with_region.csv", index=False)

print("✅ Added revenue_usd column successfully!")
print(df[["date", "region", "revenue_usd", "shipping_cost"]].head())


✅ Added revenue_usd column successfully!
         date         region  revenue_usd  shipping_cost
0  2016-01-01  North America     -5975.36        2462.18
1  2016-02-01         Europe     -5467.96        2345.89
2  2016-03-01   Asia-Pacific     -5523.72        2350.44
3  2016-04-01  Latin America     -6769.20        2611.23
4  2016-05-01  North America     -5224.76        2270.82
